# Glacier Segmentation - Dataset pipeline

This notebook contains the code that we used to create the dataset for our project.

This notebook also assumes that you have a Google Earth Engine account, ready to be used with external APIs and that you have a project where the shapefile of the glacier is hosted as a ressource (glamos/SIG_2016_wgs84).

To be ran, you will need the following libraries (not in the requirements as not needed for inference):

 - geopandas
 - shapely
 - rasterio
 - earthengine-api

 ## Library imports and configurations

In [ ]:
# Standard libraries
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil 
# Geospatial libraries
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import rasterio
from rasterio import features
import geemap
import ee

#PATHs
shape_path_2016 = "glamos/2016/SGI_2016_glaciers.shp"
sgi_2016 = gpd.read_file(shape_path_2016)

# Initialize the Earth Engine module
try:
    ee.Initialize(project="b3testdrive") # this is the name of our Google Cloud project
except Exception:
    ee.Authenticate()
    ee.Initialize(project="b3testdrive")

# Attributes of the shape file
print("available fields in the 2016 glacier inventory:", sgi_2016.columns)

Index(['gid', 'pk_glacier', 'sgi-id', 'name', 'rl_0', 'rl_1', 'rl_2', 'rl_3',
       'i_code', 'year_acq', 'year_rel', 'area_km2', 'length_km', 'masl_min',
       'masl_med', 'masl_mean', 'masl_max', 'slope_deg', 'aspect_deg',
       'geometry'],
      dtype='object')


GoogleEarthEngine **only allows uploading Shapefile in WSG84**, so we need to convert the LV95 ones!

In [ ]:
if sgi_2016.crs is None: 
    sgi_2016.set_crs(epsg=2056, inplace=True) # Ensure that it has the correct LV95 CRS
sgi_2016_wgs84 = sgi_2016.to_crs(epsg=4326) # Save a copy
sgi_2016_wgs84.to_file("glamos/SGI_2016_wgs84.shp")
print("Reprojected glacier shapefile saved to 'glamos/SGI_2016_wgs84.shp', please upload it to Earth Engine assets.")

## Downloading tiles in the Area of Interest

The system is quite simple! We have the geometry of each individual glacier, and for each of those glacier, we want to have square patches that covers them -> This is what we do here, with tiles of 224x224 of size and a resolution of 10m.

Additionally, we verify that the tile still has at list a minimum of ~50 pixels in it that are glacier

In [ ]:


# Configuration for the data acquisition
TILE_SIZE_PX = 224
RESOLUTION = 10 
TILE_SIZE_M = TILE_SIZE_PX * RESOLUTION  # 2240m
OUTPUT_DIR = "dataset/images_raw_2056" # Output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Filters
MIN_INTERSECTION_M2 = 5000 # Minimum intersection area between glacier. Arbitrary just to avoid extremly small noise (approximately ensures at leasst a 7x7 pixel area)
MIN_YEAR = 2015 # since sent2 started in mid 2015

# Loaded into EPSG:2056 (==LV95)
gdf = gpd.read_file("glamos/SGI_2016_wgs84.shp") 
if gdf.crs.to_epsg() != 2056: 
    gdf = gdf.to_crs(epsg=2056)

def get_sentinel_raw(year):
    """
    Fetch raw Sentinel-2 data (reflectance) for the given year and summer months (July to September).
    Uses median composite to reduce cloud cover.
    
    Args:
        year (int): Year of acquisition.
    """
    
    start = f'{year}-07-01'
    end = f'{year}-09-30'
    
    collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterDate(start, end) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .median() # Median to reduce clouds, Maximum 20% cloud cover
    
    # Blue, Green, Red, NIR (NIR not used in the end, but still we get it)
    return collection.select(['B2', 'B3', 'B4', 'B8'])

# Stats
stats = { "tiles_kept": 0, "errors": 0 }


print(f"Début du traitement en EPSG:2056...")

# We iterate over all the glaciers defined in the inventory shapefile
for index, row in gdf.iterrows():
    g_id = row['pk_glacier']
    year = int(row["year_acq"]) # Acquisition year -> to have an image as close as possible to the inventory date
    
    if year < MIN_YEAR: continue # Filter for Sentinel-2 availability

    # Geometry (LV95)
    geom_shapely = row.geometry
    minx, miny, maxx, maxy = geom_shapely.bounds
    
    # Grid coordinates (To cover the full size of the glacier)
    x_coords = np.arange(minx, maxx, TILE_SIZE_M)
    y_coords = np.arange(miny, maxy, TILE_SIZE_M)
    
    # Local counter = tile number for the current glacier -> we iterate over all the tiles covering the glacier
    local_cnt = 0
    for x in x_coords:
        for y in y_coords:
            # Tile definition of the box (in EPSG:2056)
            tile_box = box(x, y, x + TILE_SIZE_M, y + TILE_SIZE_M) 
            
            # Filter
            if not tile_box.intersects(geom_shapely): continue
            if tile_box.intersection(geom_shapely).area < MIN_INTERSECTION_M2: continue # Minimum intersection area explained before
            
            filename = os.path.join(OUTPUT_DIR, f"{g_id}_{year}_tile_{local_cnt}.tif")
            if os.path.exists(filename): 
                local_cnt += 1
                continue
            
            # EPSG:2056 -> EPSG:4326 (required in GEE :( )
            box_gdf = gpd.GeoSeries([tile_box], crs=2056)
            box_wgs84 = box_gdf.to_crs(epsg=4326).iloc[0]
            
            # We can use the box bounds directly to define the region in GEE
            region_ee = ee.Geometry.Rectangle(
                [box_wgs84.bounds[0], box_wgs84.bounds[1], box_wgs84.bounds[2], box_wgs84.bounds[3]],
                proj='EPSG:4326',
                geodesic=False # we want straight lines
            )
            
            try:
                # Save
                geemap.download_ee_image(
                    get_sentinel_raw(int(year)),
                    filename=filename,
                    region=region_ee,   # Area of interest
                    scale=10,           # 10m pixels
                    crs='EPSG:2056',    # EPSG:2056
                    dtype='uint16'      
                )
                stats["tiles_kept"] += 1
                local_cnt += 1
                
            except Exception as e:
                print(f"Erreur {g_id}: {e}")
                stats["errors"] += 1

print(f"Tiles generated : {stats['tiles_kept']}")
print(f"Errors encountered : {stats['errors']}")


len(gdf) = 1400
crs = EPSG:2056
total_bounds = [2552348.6644433  1078863.87018848 2827545.1764972  1235126.54123004]
example year_acq unique (head) = [2014, 2016, 2016, 2014, 2016]


## Mask Generation

Here, we want that for each of our tiles that we generated in the cell above, a new file with only the pixel belonging to glacier activated.

Additionally, we added a class_ignore index to fill the tiles: Why? Because neighbouring glacier have not always been labeled at the same year! this means that we can have a glacier that was labeled in 2016 on the left of our image, and a glacier labeled in 2018 on the right, where the boundaries are not the same as before! Thus, when we generate masks on a tile, if there is a difference in more than 1 year, we fill the other glacier with 255 -> ignored by the model (so it doesn't get punished or rewared on them)

In [ ]:
# Paths
IMAGE_DIR = "dataset/images_raw_2056"
MASK_DIR = "dataset/masks"
SHP_PATH = "glamos/SGI_2016_wgs84.shp"

os.makedirs(MASK_DIR, exist_ok=True)

# Year tolerance
YEAR_TOLERANCE = 1 

# Classes definitions
CLASS_BACKGROUND = 0
CLASS_GLACIER = 1
CLASS_IGNORE = 255

# Project shapefile to EPSG:2056 (LV95)
gdf = gpd.read_file(SHP_PATH)
gdf = gdf.set_crs(epsg=4326, allow_override=True).to_crs(epsg=2056)
if gdf.crs.to_epsg() != 2056:
    gdf = gdf.to_crs(epsg=2056)

gdf["pk_glacier"] = gdf["pk_glacier"].astype(str)

# Spatial index for fast queries
sindex = gdf.sindex

# List all the generated images
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith('.tif')]
total = len(image_files)
print(f"Generating masks for {total} images")

for filename in tqdm(image_files):
    
    # Based on the filename we stored, we can recover the glacier ID and the year
    try:
        parts = filename.replace('.tif', '').split('_')
        # format : ID_COMPLEX_YEAR_tile_NUMBER
        target_year = int(parts[-3]) 
        target_id = "_".join(parts[:-3])
    except Exception as e:
        print(f"   Filename error: {filename} -> {e}")
        continue
    
    # The paths are "symmetric" to the images     
    image_path = os.path.join(IMAGE_DIR, filename)
    mask_path = os.path.join(MASK_DIR, filename) 
    
    with rasterio.open(image_path) as src:
        # Spatial info
        out_shape = (src.height, src.width) # Expected (224, 224)
        transform = src.transform 
        bounds = src.bounds 
        
        # We recover the bounding box of the image, which we will then use to filter glaciers
        bbox_geom = box(bounds.left, bounds.bottom, bounds.right, bounds.top)
        
        # Filter by spatial index (fast due to the spatail index that was built)
        possible_matches_index = list(sindex.query(bbox_geom))
        possible_matches = gdf.iloc[possible_matches_index]
        
        # Filter by intersection
        visible_glaciers = possible_matches[possible_matches.intersects(bbox_geom)].copy()
        
        # Empty mask (full black image by default)
        mask = np.zeros(out_shape, dtype=np.uint8)

        if not visible_glaciers.empty:
            # Check acquisition year
            visible_glaciers['year_diff'] = (visible_glaciers['year_acq'] - target_year).abs()
            
            is_target = visible_glaciers['pk_glacier'].astype(str) == str(target_id)
            is_good_year = visible_glaciers['year_diff'] <= YEAR_TOLERANCE
            
            valid_mask = is_target | is_good_year # target glacier or neighbor within tolerance
            
            bad_glaciers = visible_glaciers[~valid_mask]  # check neighbors too old/recent
            good_glaciers = visible_glaciers[valid_mask]  # target year + recent neighbors
            
            # Rasterisation with the ignore class (turning the geometries into raster masks)
            if not bad_glaciers.empty:
                shapes_bad = ((geom, CLASS_IGNORE) for geom in bad_glaciers.geometry)
                features.rasterize(
                    shapes=shapes_bad,
                    out=mask,
                    transform=transform,
                    default_value=CLASS_IGNORE,
                    dtype=np.uint8
                )
                
            # Rasterisation with the glacier that match the criteria
            if not good_glaciers.empty:
                shapes_good = ((geom, CLASS_GLACIER) for geom in good_glaciers.geometry)
                features.rasterize(
                    shapes=shapes_good,
                    out=mask,
                    transform=transform,
                    default_value=CLASS_GLACIER,
                    dtype=np.uint8
                )
        
        # Save just 1 band, in UINT8
        meta = src.meta.copy()
        meta.update({
            "count": 1,
            "dtype": 'uint8',
            "driver": "GTiff",
            "compress": "lzw",
            "nodata": 0 # back = 0
        })
        
        with rasterio.open(mask_path, 'w', **meta) as dst:
            dst.write(mask, 1)

## Corrupted Images (fully black)
Some images resulted in being fully black! We filter them out of the dataset with the following cell 

In [ ]:

SRC_IMG_DIR = "dataset/images_raw_2056"
SRC_MASK_DIR = "dataset/masks"

CLEAN_IMG_DIR = "dataset/clean/images"
CLEAN_MASK_DIR = "dataset/clean/masks"

os.makedirs(CLEAN_IMG_DIR, exist_ok=True)
os.makedirs(CLEAN_MASK_DIR, exist_ok=True)

files = [f for f in os.listdir(SRC_IMG_DIR) if f.endswith('.tif')]
corrupted_count = 0
valid_count = 0

# Clean process

for f in tqdm(files):
    src_img_path = os.path.join(SRC_IMG_DIR, f)
    src_mask_path = os.path.join(SRC_MASK_DIR, f)
    
    # Check mask existence
    if not os.path.exists(src_mask_path):
        corrupted_count += 1
        continue

    is_valid = False
    
    # Check contents (not empty)
    try:
        with rasterio.open(src_img_path) as src:
            
            if src.read().max() > 0:
                is_valid = True
    except Exception as e:
        print(f"Error {f}: {e}")
    
    if is_valid:
        shutil.copy2(src_img_path, os.path.join(CLEAN_IMG_DIR, f))
        shutil.copy2(src_mask_path, os.path.join(CLEAN_MASK_DIR, f))
        valid_count += 1
    else:
        corrupted_count += 1


print(f"Rejected images : {corrupted_count}")
print(f"Valid images : {valid_count}")


Half of the data is gone, but this is the cost of a high quality dataset. To visualize what's left of the data, here we filter the original shapefile with only the glacier for which we have some images. (Only useful for visualization)

In [ ]:
CLEAN_IMG_DIR = "dataset/clean/images"
ORIGINAL_SHP = "glamos/SGI_2016_wgs84.shp"
OUTPUT_SHP = "glamos/SGI_2016_VALID_ONLY.shp"


files = [f for f in os.listdir(CLEAN_IMG_DIR) if f.endswith('.tif')]
if not files:
    raise ValueError("Erreur : Le dossier d'images est vide !")

# IDs extraction - expected format : ID_YEAR_tile_X.tif
valid_ids = set()
for f in tqdm(files):
    parts = f.replace('.tif', '').split('_')
    if len(parts) >= 3:
        g_id = "_".join(parts[:-3])
        valid_ids.add(g_id)
print(f" Unique IDs: {len(valid_ids)} ")

gdf = gpd.read_file(ORIGINAL_SHP)
# Convert ID into string
gdf['pk_glacier'] = gdf['pk_glacier'].astype(str)
# Filter
gdf_valid = gdf[gdf['pk_glacier'].isin(valid_ids)].copy()
print(f" {len(gdf_valid)} polygones kept over {len(gdf)} ")
# Save
gdf_valid.to_file(OUTPUT_SHP)
print("Filtered shapefile saved to", OUTPUT_SHP,"Can be used on QGIS to see which of our Glacier are not trown out.")


## Test Set isolation
Using the new shapefile that the previous cell generated, we used QGIS to create a list of glacier (spacially isolated) to be used as a test set, that we will move in another folder here.

In [ ]:
DRY_RUN = False  # small security just to do the checks
TSV_PATH = "dataset/test_set_idx.tsv"
SRC_IMG_DIR = "dataset/clean/images"
SRC_MASK_DIR = "dataset/clean/masks"
DEST_IMG_DIR = "dataset/test/images"
DEST_MASK_DIR = "dataset/test/masks"

if not DRY_RUN:
    os.makedirs(DEST_IMG_DIR, exist_ok=True)
    os.makedirs(DEST_MASK_DIR, exist_ok=True)

# We retreive the IDs of the glacier using the TSV that we exported from QGIS
test_set = pd.read_csv(TSV_PATH, sep="\t")
target_ids = set(test_set["pk_glacier"].astype(str).str.strip().str.replace('"', ''))
files = [f for f in os.listdir(SRC_IMG_DIR) if f.endswith('.tif')]


files_to_move = []
ids_found_in_files = set()
# We just do a greedy match 
for filename in tqdm(files, desc="Matching test set IDs"):
    parts = filename.replace('.tif', '').split('_')
    if len(parts) >= 3:
        file_glacier_id = "_".join(parts[:-3])
        # ID found in target??
        if file_glacier_id in target_ids:
            files_to_move.append(filename)
            ids_found_in_files.add(file_glacier_id)

#Compute stats
missing_ids = target_ids - ids_found_in_files
total_tiles = len(files_to_move)

print(f"Results Dry Run simulation: {DRY_RUN})")
print(f"Target : {len(target_ids)}")
print(f"Glaciers found: {len(ids_found_in_files)}")
print(f"Missing glaciers: {len(missing_ids)}")
print(f"Total tiles: {total_tiles}")

if len(missing_ids) > 0:
    print(f"Examples of missing IDs: {list(missing_ids)[:5]}")
    print("Glaciers found in target but no matching images in clean")

if total_tiles > 0:
    if DRY_RUN:
        print(f"\ntest: {total_tiles} files")
    else:
        moved_count = 0
        for filename in tqdm(files_to_move, desc="Déplacement"):
            src_img = os.path.join(SRC_IMG_DIR, filename)
            src_msk = os.path.join(SRC_MASK_DIR, filename)
            
            dst_img = os.path.join(DEST_IMG_DIR, filename)
            dst_msk = os.path.join(DEST_MASK_DIR, filename)
            
            try:
                shutil.move(src_img, dst_img)
                
                if os.path.exists(src_msk):
                    shutil.move(src_msk, dst_msk)
                moved_count += 1
            except Exception as e:
                print(f"Error in {filename}: {e}")
                
        print(f"\n{moved_count} tiles in dataset/test")
else:
    print("\nNothing to move")